# Usando a ResNet50 para treinar com data augmentation

Data Aug: https://www.tensorflow.org/tutorials/images/data_augmentation </br>
https://www.tensorflow.org/api_docs/python/tf/keras/layers

ResNet:
https://www.tensorflow.org/api_docs/python/tf/keras/applications/resnet50/ResNet50 </br>
https://chroniclesofai.com/transfer-learning-with-keras-resnet-50/

In [ ]:
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout, Rescaling
from tensorflow.keras import optimizers
from tensorflow.keras.utils import image_dataset_from_directory

!pip install pydicom
import numpy as np
import pydicom
from PIL import Image
import os

import cv2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 33.8 MB/s 


In [ ]:
!mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

# Ajustando o DS

## Baixando o DS

In [ ]:
!kaggle datasets download -d pedroamaro/smaller-rsna-ds-train-test

100% 7.24G/7.25G [03:26<00:00, 42.7MB/s]
100% 7.25G/7.25G [03:26<00:00, 37.7MB/s]


In [ ]:
!unzip /content/smaller-rsna-ds-train-test.zip | awk 'BEGIN {ORS=" "} {if(NR%1000==0)print "."}'
!rm /content/smaller-rsna-ds-train-test.zip

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

In [ ]:
print('Numero de imagens: NO')
!ls /content/small_rsna_ds_train/no | wc -l
print('Numero de imagens: YES')
!ls /content/small_rsna_ds_train/yes | wc -l

Numero de imagens: NO
20047
Numero de imagens: YES
3287


## Salvando como JPG e aplicando full windowing

In [ ]:
!mkdir small_rsna_ds_as_jpg
!mkdir /content/small_rsna_ds_as_jpg/no
!mkdir /content/small_rsna_ds_as_jpg/yes

!mkdir test_as_jpg
!mkdir test_as_jpg/no
!mkdir test_as_jpg/yes

In [ ]:
def segment_circle(windowed):
    original = windowed.copy().astype("uint8")
    mask = np.zeros(original.shape, dtype=np.uint8)
    gray = original
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
    close = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=5)
    cnts = cv2.findContours(close, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    for c in cnts:
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.04 * peri, True)
        area = cv2.contourArea(c)
        if len(approx) > 4 and area > 10000 and area < 500000:
            ((x, y), r) = cv2.minEnclosingCircle(c)
            cv2.circle(mask, (int(x), int(y)), int(r), (255, 255, 255), -1)
            cv2.circle(original, (int(x), int(y)), int(r), (36, 255, 12), 0)
    x,y,w,h = cv2.boundingRect(mask)
    mask_ROI = mask[y:y+h, x:x+w]
    image_ROI = original[y:y+h, x:x+w]
    result = cv2.bitwise_and(image_ROI, image_ROI, mask=mask_ROI)
    return result, (x, y, w, h)

def correct_dcm(dcm):
    x = dcm.pixel_array + 1000
    px_mode = 4096
    x[x>=px_mode] = x[x>=px_mode] - px_mode
    dcm.PixelData = x.tobytes()
    dcm.RescaleIntercept = -1000

def window_image(img, window_center, window_width):
    img_min = window_center - window_width // 2
    img_max = window_center + window_width // 2
    img = np.clip(img, img_min, img_max)
    return img


def preprocess_and_segment(path, make_correct=True):    
    dcm = pydicom.dcmread(path)
    if make_correct and (dcm.BitsStored == 12) and (dcm.PixelRepresentation == 0) and (int(dcm.RescaleIntercept) > -100):
        correct_dcm(dcm)

    sample = dcm.pixel_array.astype("float32") * dcm.RescaleSlope + dcm.RescaleIntercept

    brain_window = window_image(sample, 40, 80)
    dural_window = window_image(sample, 80, 200)
    bone_window = window_image(sample, 600, 2800)

    brain_window, (x,y,w,h) = segment_circle(brain_window)
    if brain_window is None:
        return np.zeros((512, 512, 3))
    dural_window = dural_window[y:y+h, x:x+w]
    bone_window = bone_window[y:y+h, x:x+w]
    
    brain_window = (brain_window - (0.)) / 80.
    dural_window = (dural_window - (-20.)) / 200.
    bone_window = (bone_window - (-1200.)) / 2800.
    img_3ch = np.dstack([brain_window, dural_window, bone_window]).astype("float32")

    return img_3ch


def save_W_as_jpg(path, id, dest_folder):

  im = preprocess_and_segment(path)

  rescaled_im = (np.maximum(im, 0)/im.max())*255.0
  final_im = np.uint8(rescaled_im)

  final_im = Image.fromarray(final_im)
  final_im.save(dest_folder+id+'.jpg')

### Para o conjunto Train

In [ ]:
train_path = "/content/small_rsna_ds_train/yes/"
folder = os.listdir("/content/small_rsna_ds_train/yes/")
dest_folder = "/content/small_rsna_ds_as_jpg/yes/"

for id_ in folder:
  img_id = id_[:12]
  save_W_as_jpg(train_path + f"{id_}", img_id, dest_folder)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:67: RuntimeWarning: invalid value encountered in true_divide


In [ ]:
train_path = "/content/small_rsna_ds_train/no/"
folder = os.listdir("/content/small_rsna_ds_train/no/")
dest_folder = "/content/small_rsna_ds_as_jpg/no/"

for id_ in folder:
  img_id = id_[:12]
  save_W_as_jpg(train_path + f"{id_}", img_id, dest_folder)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:67: RuntimeWarning: invalid value encountered in true_divide


In [ ]:
print('Numero de imagens: NO')
!ls /content/small_rsna_ds_as_jpg/no/ | wc -l
print('Numero de imagens: YES')
!ls /content/small_rsna_ds_as_jpg/yes/ | wc -l

Numero de imagens: NO
20047
Numero de imagens: YES
3287


### Para o conjunto Test

In [ ]:
train_path = "/content/small_rsna_ds_test/yes/"
folder = os.listdir("/content/small_rsna_ds_test/yes/")
dest_folder = "/content/test_as_jpg/yes/"

for id_ in folder:
  img_id = id_[:12]
  save_W_as_jpg(train_path + f"{id_}", img_id, dest_folder)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:67: RuntimeWarning: invalid value encountered in true_divide


In [ ]:
train_path = "/content/small_rsna_ds_test/no/"
folder = os.listdir("/content/small_rsna_ds_test/no/")
dest_folder = "/content/test_as_jpg/no/"

for id_ in folder:
  img_id = id_[:12]
  save_W_as_jpg(train_path + f"{id_}", img_id, dest_folder)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:67: RuntimeWarning: invalid value encountered in true_divide


In [ ]:
print('Numero de imagens: NO')
!ls /content/test_as_jpg/no/ | wc -l
print('Numero de imagens: YES')
!ls /content/test_as_jpg/yes/ | wc -l

Numero de imagens: NO
10022
Numero de imagens: YES
1643


# Carregando o dataset

In [ ]:
train_data = image_dataset_from_directory(
    "/content/small_rsna_ds_as_jpg",
     labels='inferred',
     validation_split=1/3,
     subset="training",
     seed=123,
     batch_size = 64,
     color_mode = "rgb"
     )

Found 23334 files belonging to 2 classes.
Using 15556 files for training.


In [ ]:
val_data = image_dataset_from_directory(
    "/content/small_rsna_ds_as_jpg",
     labels='inferred',
     validation_split=1/3,
     subset="validation",
     seed=123,
     batch_size = 64,
     color_mode = "rgb"
)

Found 23334 files belonging to 2 classes.
Using 7778 files for validation.


In [ ]:
test_data = image_dataset_from_directory(
    "/content/test_as_jpg",
     labels='inferred',
     label_mode="int",
     batch_size = 64,
     color_mode = "rgb"
     )

Found 11665 files belonging to 2 classes.


In [ ]:
class_names = train_data.class_names

## Normalizando e Pegando o tamanho de entrada

In [ ]:
normalization_layer = Rescaling(1./255)

normalized_data = train_data.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_data))

normalized_val_data = val_data.map(lambda x, y: (normalization_layer(x), y))
val_image_batch, val_labels_batch = next(iter(normalized_val_data))

normalized_test_data = test_data.map(lambda x, y: (normalization_layer(x), y))
test_image_batch, test_labels_batch = next(iter(normalized_test_data))

# ResNet com data aug

In [ ]:
from keras.layers import RandomFlip, RandomRotation, RandomCrop

In [ ]:
keras.backend.clear_session()

In [ ]:
resnet_model = Sequential()

resnet_model.add(RandomFlip("horizontal"))
resnet_model.add(RandomRotation(0.2)) #que valor usar?
resnet_model.add(RandomCrop(256, 256, seed=1)) #height e width do output e a seed

resnet = tf.keras.applications.resnet50.ResNet50(
    include_top=False,
    input_shape=image_batch.shape[1:],
    pooling='avg',
    weights='imagenet',
    classes=2,
)

resnet_model.add(resnet)

resnet_model.add(Flatten())

resnet_model.add(Dense(512, activation="relu"))

resnet_model.add(Dropout(0.5))

resnet_model.add(Dense(1, activation="sigmoid"))

In [ ]:
neg = 20047
pos = 3287
total = pos+neg

w0 = (1 / neg) * (total / 2.0)
w1 = (1 / pos) * (total / 2.0)

print('Peso para a classe 0: {:.2f}'.format(w0))
print('Peso para a classe 1: {:.2f}'.format(w1))

Peso para a classe 0: 0.58
Peso para a classe 1: 3.55


In [ ]:
resnet_model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = resnet_model.fit(normalized_data, validation_data=normalized_val_data, epochs=5, class_weight={0:w0, 1:w1}, 
  callbacks=[keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/5
244/244 [==============================] - 250s 926ms/step - loss: 0.6261 - auc: 0.7236 - accuracy: 0.6627 - val_loss: 0.4970 - val_auc: 0.3842 - val_accuracy: 0.8579
Epoch 2/5
244/244 [==============================] - 227s 929ms/step - loss: 0.5151 - auc: 0.8235 - accuracy: 0.7540 - val_loss: 0.4549 - val_auc: 0.5586 - val_accuracy: 0.8541
Epoch 3/5
244/244 [==============================] - 227s 927ms/step - loss: 0.4689 - auc: 0.8545 - accuracy: 0.8022 - val_loss: 0.4572 - val_auc: 0.6893 - val_accuracy: 0.8116
Epoch 4/5
244/244 [==============================] - 228s 930ms/step - loss: 0.4353 - auc: 0.8738 - accuracy: 0.8282 - val_loss: 0.5379 - val_auc: 0.8332 - val_accuracy: 0.7828
Epoch 5/5
244/244 [==============================] - 227s 928ms/step - loss: 0.4154 - auc: 0.8865 - accuracy: 0.8401 - val_loss: 0.4498 - val_auc: 0.9025 - val_accuracy: 0.8312


In [ ]:
results = resnet_model.evaluate(normalized_test_data)
print(results)

183/183 [==============================] - 50s 269ms/step - loss: 0.4626 - auc: 0.8990 - accuracy: 0.8267
[0.46262094378471375, 0.8989992141723633, 0.8266609311103821]


## mais epochs

In [ ]:
keras.backend.clear_session()

In [ ]:
resnet_model = Sequential()

resnet_model.add(RandomFlip("horizontal"))
resnet_model.add(RandomRotation(0.2)) #que valor usar?
resnet_model.add(RandomCrop(256, 256, seed=1)) #height e width do output e a seed

resnet = tf.keras.applications.resnet50.ResNet50(
    include_top=False,
    input_shape=image_batch.shape[1:],
    pooling='avg',
    weights='imagenet',
    classes=2,
)

resnet_model.add(resnet)

resnet_model.add(Flatten())

resnet_model.add(Dense(512, activation="relu"))

resnet_model.add(Dropout(0.5))

resnet_model.add(Dense(1, activation="sigmoid"))

In [ ]:
neg = 20047
pos = 3287
total = pos+neg

w0 = (1 / neg) * (total / 2.0)
w1 = (1 / pos) * (total / 2.0)

print('Peso para a classe 0: {:.2f}'.format(w0))
print('Peso para a classe 1: {:.2f}'.format(w1))

Peso para a classe 0: 0.58
Peso para a classe 1: 3.55


In [ ]:
resnet_model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = resnet_model.fit(normalized_data, validation_data=normalized_val_data, epochs=20, class_weight={0:w0, 1:w1}, 
  callbacks=[keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/20
244/244 [==============================] - 235s 933ms/step - loss: 0.6127 - auc: 0.7358 - accuracy: 0.6787 - val_loss: 0.4579 - val_auc: 0.6754 - val_accuracy: 0.8581
Epoch 2/20
244/244 [==============================] - 227s 928ms/step - loss: 0.5060 - auc: 0.8294 - accuracy: 0.7654 - val_loss: 0.3985 - val_auc: 0.6966 - val_accuracy: 0.8592
Epoch 3/20
244/244 [==============================] - 228s 933ms/step - loss: 0.4535 - auc: 0.8633 - accuracy: 0.8130 - val_loss: 0.4356 - val_auc: 0.6762 - val_accuracy: 0.8384
Epoch 4/20
244/244 [==============================] - 227s 930ms/step - loss: 0.4263 - auc: 0.8811 - accuracy: 0.8374 - val_loss: 0.5285 - val_auc: 0.8339 - val_accuracy: 0.7780
Epoch 5/20
244/244 [==============================] - 228s 932ms/step - loss: 0.3995 - auc: 0.8958 - accuracy: 0.8499 - val_loss: 0.3840 - val_auc: 0.9023 - val_accuracy: 0.8629
Epoch 6/20
244/244 [==============================] - 227s 929ms/step - loss: 0.3865 - auc: 0.9023 - accuracy:

In [ ]:
results = resnet_model.evaluate(normalized_test_data)
print(results)

183/183 [==============================] - 49s 266ms/step - loss: 0.3533 - auc: 0.9076 - accuracy: 0.8764
[0.3533322811126709, 0.9075692892074585, 0.8763823509216309]


# ResNet sem data aug

In [ ]:
keras.backend.clear_session()

In [ ]:
resnet_model = Sequential()

resnet = tf.keras.applications.resnet50.ResNet50(
    include_top=False,
    input_shape=image_batch.shape[1:],
    pooling='avg',
    weights='imagenet',
    classes=2,
)

resnet_model.add(resnet)

resnet_model.add(Flatten())

resnet_model.add(Dense(512, activation="relu"))

resnet_model.add(Dropout(0.5))

resnet_model.add(Dense(1, activation="sigmoid"))

In [ ]:
neg = 20047
pos = 3287
total = pos+neg

w0 = (1 / neg) * (total / 2.0)
w1 = (1 / pos) * (total / 2.0)

print('Peso para a classe 0: {:.2f}'.format(w0))
print('Peso para a classe 1: {:.2f}'.format(w1))

Peso para a classe 0: 0.58
Peso para a classe 1: 3.55


In [ ]:
resnet_model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = resnet_model.fit(normalized_data, validation_data=normalized_val_data, epochs=5, class_weight={0:w0, 1:w1}, 
  callbacks=[keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

In [ ]:
results = resnet_model.evaluate(normalized_test_data)
print(results)

## mais epochs

In [ ]:
keras.backend.clear_session()

In [ ]:
resnet_model = Sequential()

resnet = tf.keras.applications.resnet50.ResNet50(
    include_top=False,
    input_shape=image_batch.shape[1:],
    pooling='avg',
    weights='imagenet',
    classes=2,
)

resnet_model.add(resnet)

resnet_model.add(Flatten())

resnet_model.add(Dense(512, activation="relu"))

resnet_model.add(Dropout(0.5))

resnet_model.add(Dense(1, activation="sigmoid"))

In [ ]:
neg = 20047
pos = 3287
total = pos+neg

w0 = (1 / neg) * (total / 2.0)
w1 = (1 / pos) * (total / 2.0)

print('Peso para a classe 0: {:.2f}'.format(w0))
print('Peso para a classe 1: {:.2f}'.format(w1))

Peso para a classe 0: 0.58
Peso para a classe 1: 3.55


In [ ]:
resnet_model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = resnet_model.fit(normalized_data, validation_data=normalized_val_data, epochs=20, class_weight={0:w0, 1:w1}, 
  callbacks=[keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/20
244/244 [==============================] - 233s 929ms/step - loss: 0.5850 - auc: 0.7650 - accuracy: 0.6997 - val_loss: 0.4650 - val_auc: 0.3666 - val_accuracy: 0.8581
Epoch 2/20
244/244 [==============================] - 226s 924ms/step - loss: 0.4344 - auc: 0.8763 - accuracy: 0.8192 - val_loss: 0.4268 - val_auc: 0.5242 - val_accuracy: 0.8581
Epoch 3/20
244/244 [==============================] - 225s 921ms/step - loss: 0.3347 - auc: 0.9296 - accuracy: 0.8777 - val_loss: 0.4048 - val_auc: 0.6560 - val_accuracy: 0.8645
Epoch 4/20
244/244 [==============================] - 226s 924ms/step - loss: 0.2537 - auc: 0.9609 - accuracy: 0.9181 - val_loss: 0.4546 - val_auc: 0.8195 - val_accuracy: 0.8407
Epoch 5/20
244/244 [==============================] - 226s 923ms/step - loss: 0.1771 - auc: 0.9804 - accuracy: 0.9528 - val_loss: 0.3963 - val_auc: 0.8638 - val_accuracy: 0.8815
Epoch 6/20
244/244 [==============================] - 226s 923ms/step - loss: 0.1428 - auc: 0.9858 - accuracy:

In [ ]:
results = resnet_model.evaluate(normalized_test_data)
print(results)

183/183 [==============================] - 49s 263ms/step - loss: 0.3954 - auc: 0.8729 - accuracy: 0.8797
[0.3954440951347351, 0.8728808164596558, 0.8797256946563721]
